In [79]:
from pathlib import Path

dataset_files = Path('./data/').glob('**/*.md')
dataset_files = list(dataset_files)
dataset_files[:-1]

[PosixPath('data/pobreza.md'),
 PosixPath('data/inflacion.md'),
 PosixPath('data/mineria.md'),
 PosixPath('data/emisiones-de-gases-de-efecto-invernadero.md'),
 PosixPath('data/ciencia-y-tecnologia.md'),
 PosixPath('data/desarrollo-humano.md'),
 PosixPath('data/estructura-productiva.md'),
 PosixPath('data/servicios-basados-en-el-conocimiento.md'),
 PosixPath('data/crecimiento.md'),
 PosixPath('data/informalidad-y-desempleo.md'),
 PosixPath('data/transicion-energetica.md'),
 PosixPath('data/cambio-climatico.md'),
 PosixPath('data/comercio-exterior.md'),
 PosixPath('data/demografia.md'),
 PosixPath('data/desigualdad.md'),
 PosixPath('data/agroindustria.md'),
 PosixPath('data/gasto-publico.md'),
 PosixPath('data/trabajo-y-participacion-laboral.md'),
 PosixPath('data/salarios-e-ingresos.md')]

In [80]:
# Si vas a detectar encodings automáticamente, instala primero:
# !pip install charset-normalizer

from pathlib import Path
from typing import Tuple, Optional

def read_text_safely(path: Path, try_encodings=None, use_detector=True) -> Tuple[str, Optional[str]]:
    """
    Lee texto probando varias codificaciones. Devuelve (contenido, encoding_usada).
    """
    if try_encodings is None:
        # orden razonable para Markdown/CSV en ES
        try_encodings = ["utf-8", "utf-8-sig", "cp1252", "latin-1"]

    # 1) intentos directos
    for enc in try_encodings:
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                return f.read(), enc
        except Exception:
            pass

    # 2) detección automática (charset-normalizer)
    if use_detector:
        try:
            from charset_normalizer import from_path
            res = from_path(path).best()
            if res is not None:
                return str(res), res.encoding
        except Exception:
            pass

    # 3) último recurso: decodificar bytes con reemplazo para no reventar
    with open(path, "rb") as fb:
        raw = fb.read()
    try:
        return raw.decode("utf-8", errors="replace"), "utf-8 (replace)"
    except Exception:
        return raw.decode("latin-1", errors="replace"), "latin-1 (replace)"


# ---- Uso con tu lista dataset_files ----
if dataset_files:
    first_file_path = Path(dataset_files[0])
    try:
        content, used_enc = read_text_safely(first_file_path)
        print(f"Encoding usada: {used_enc}")
        print(f"Content of {first_file_path}:\n")
        print(content[:500])  # preview
    except Exception as e:
        print(f"Error leyendo {first_file_path}: {e}")
else:
    print("No files found in the previous step.")

data = []
for file in dataset_files:
    path = Path(file)
    try:
        file_content, used_enc = read_text_safely(path)
        data.append(file_content)
        print(f"Leído {path} con encoding {used_enc}, {len(file_content)} caracteres.")
    except Exception as e:
        print(f"Error leyendo {path}: {e}")


Encoding usada: utf-8
Content of data/pobreza.md:

## Conceptos básicos

Conceptualmente, la pobreza implica estar por debajo de un umbral mínimo de bienestar. Pero, ¿cómo operativizar este concepto abstracto en un indicador concreto? Existen diversas metodologías para hacerlo y así medir la pobreza.

La más difundida es la pobreza monetaria (o por ingresos), que mide el bienestar de los hogares por su nivel de ingreso. La situación de pobreza se determina al comparar ese ingreso con un valor monetario: la línea de pobreza. Esta indica el monto 
Leído data/pobreza.md con encoding utf-8, 47830 caracteres.
Leído data/inflacion.md con encoding utf-8, 23250 caracteres.
Leído data/mineria.md con encoding utf-8, 36156 caracteres.
Leído data/emisiones-de-gases-de-efecto-invernadero.md con encoding utf-8, 41476 caracteres.
Leído data/ciencia-y-tecnologia.md con encoding utf-8, 41656 caracteres.
Leído data/desarrollo-humano.md con encoding utf-8, 43317 caracteres.
Leído data/estructura-producti

In [81]:
import re
import unicodedata

def clean_for_rag(text: str) -> str:
    """
    Limpia texto semiestructurado (Markdown/HTML ligero) para RAG.
    - Elimina secuencias literales \n y saltos reales de línea
    - Quita URLs
    - Remueve etiquetas sueltas (PNG, JPG, GIF) y bloques [CSV](...)
    - Despoja marcas Markdown (#, ###, **, _) conservando el texto
    - Quita restos de código/markup comunes
    - Normaliza espacios y puntuación básica
    """
    # 0) Normaliza Unicode (acentos, etc.)
    t = unicodedata.normalize("NFC", text)

    # 1) Convierte literales "\n" en saltos reales y luego a espacios
    t = t.replace("\\n", "\n")

    # 2) Elimina URLs
    t = re.sub(r'https?://\S+', '', t)

    # 3) Quita bloques tipo [Texto](enlace) -> conserva "Texto"
    t = re.sub(r'\[([^\]]+)\]\((?:[^)]+)\)', r'\1', t)

    # 4) Borra etiquetas de imagen/formatos sueltas (PNG, JPG, GIF, PDF) como palabras
    t = re.sub(r'\b(PNG|JPG|JPEG|GIF|PDF|SVG)\b', '', t, flags=re.IGNORECASE)

    # 5) Quita encabezados Markdown preservando el texto
    #    "# Título" -> "Título", también "### Subtítulo"
    t = re.sub(r'(^|\n)\s{0,3}#{1,6}\s*', r'\1', t)

    # 6) Quita énfasis Markdown **negrita** y _itálica_ conservando contenido
    t = t.replace('**', '').replace('__', '')
    t = re.sub(r'(?<!\w)[*_](.*?)[*_](?!\w)', r'\1', t)

    # 7) Quita guiones/listas sin contenido estructural
    t = re.sub(r'(^|\n)\s*[-•*]\s*', r'\1', t)

    # 8) Quita restos de etiquetas/artefactos tipo "Nota:" si vienen pegados a markup
    #    (si quieres conservar "Nota:" comenta esta línea)
    # t = re.sub(r'\bNota:\s*', '', t, flags=re.IGNORECASE)

    # 9) Colapsa saltos múltiples a un solo espacio
    t = re.sub(r'[\r\n]+', ' ', t)

    # 10) Espacios antes de puntuación y dobles espacios
    t = re.sub(r'\s+([,;:.!?])', r'\1', t)
    t = re.sub(r'\s{2,}', ' ', t).strip()

    return t

clean_data = []
for content in data:
    clean_data.append(clean_for_rag(content))


In [82]:
from splitter import RecursiveCharacterSplitter

splitter = RecursiveCharacterSplitter(
    chunk_size=400,
    chunk_overlap=200
)

In [83]:
from common import Metadata
from uuid import uuid4

ids = list[str]()
chunks = list[str]()
metadatas = list[dict]()

for file in dataset_files:
    file = Path(file)

    clean_file = clean_for_rag(file.read_text(encoding='utf8'))


    for chunk in splitter.split_text(clean_file):
        ids.append(str(uuid4()))

        chunks.append(chunk)

        path = str(file.resolve())
        path = path.replace("\\", "/").split("/")[-1]
        metadata = Metadata(path=path)
        metadata = metadata.model_dump(mode='json', exclude_none=True)
        metadatas.append(metadata)

In [84]:
from sentence_transformers import SentenceTransformer

modelo_embeddings = SentenceTransformer(
    model_name_or_path="sentence-transformers/all-mpnet-base-v2",
    device="cuda", # "cpu"
    backend='torch'
)

In [85]:
vectores = modelo_embeddings.encode(chunks, show_progress_bar=True)

Batches:   0%|          | 0/98 [00:00<?, ?it/s]

In [86]:
import chromadb

In [87]:
db = chromadb.PersistentClient(path="chroma")

In [89]:
db.delete_collection('argendata_topicos')

In [90]:
coleccion = db.create_collection("argendata_topicos")

In [91]:
coleccion.add(
    ids=ids,
    documents=chunks,
    embeddings=vectores,
    metadatas=metadatas,
)

In [92]:
result = coleccion.query(modelo_embeddings.encode(["La pobreza de argentina disminuyo un 10%"]),
                n_results=5)

In [124]:
from google.colab import userdata
from google import genai
import json

api_key = userdata.get('GOOGLE_API_KEY') or userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=api_key)  # el cliente toma la key aquí

def check_truth(query):

  result = coleccion.query(modelo_embeddings.encode([query]),
                  n_results=30)

  docs = (result.get("documents") or [[]])[0]
  metas = (result.get("metadatas") or [[]])[0]

  input_llm = [
      {"path": (md or {}).get("path", ""), "content": doc}
      for doc, md in zip(docs, metas)
  ]

  context = "\n".join([f"File: {pair['path']}\nContent: {pair['content']}" for pair in input_llm])

  prompt = (
      """
      TAREA (RAG):
      Recibirás una SENTENCIA (afirmación) y un CONTEXTO formado por fragmentos recuperados de documentos (con sus paths).
      Debes evaluar la SENTENCIA usando EXCLUSIVAMENTE ese CONTEXTO. Tu objetivo es clasificar la afirmación y fundamentar la respuesta
      con las fuentes (paths) recuperadas. Si el CONTEXTO no es concluyente, indícalo.

      INSTRUCCIONES (NO REVELAR):
      - Usa EXCLUSIVAMENTE el Contexto provisto. No inventes datos.
      - Determina si la Sentencia es: "verdad", "falso" o "no hay informacion suficiente".
      - Si el contexto no confirma con claridad la Sentencia, responde "no hay informacion suficiente".
      - RESPONDE EXCLUSIVAMENTE con un JSON VÁLIDO. No incluyas ningún texto adicional, ni explicaciones fuera del JSON, ni bloques de código, ni comentarios.
      - Usa comillas dobles en claves y valores. No uses comas finales (trailing commas).
      - El JSON debe seguir EXACTAMENTE este esquema:
      {
        "Resultado": "verdad|falso|NA",
        "Respuesta": "<explicación breve y directa citando el dato del contexto (si aplica)>",
        "paths": ["<path1>", "<path2>", "..."]
      }
      - "paths" debe listar hasta 3 archivos del contexto que fundamenten la respuesta (sin duplicados).

      SENTENCIA:
      {sentencia}

      CONTEXTO:
      {contexto}
      """
      f"Sentencia: {query}\n\n"
      f"Contexto:\n{context}"
  )

  resp = client.models.generate_content(
      model="gemini-2.5-flash",   # modelo estable recomendado
      contents=prompt
  )

  resp = resp.text

  if resp.startswith("```"):
      resp = re.sub(r"^```(?:json)?\s*|\s*```$", "", resp, flags=re.S).strip()

  data = json.loads(resp)

  return data


In [125]:
query = 'La pobreza se redujo hacia fines de 1993'

check_truth(query)

{'Resultado': 'verdad',
 'Respuesta': 'En los primeros años del régimen de convertibilidad (1991-2001), la pobreza se redujo y alcanzó el 27% hacia fines de 1993. Además, el nivel de pobreza de aproximadamente 26% en el segundo semestre de 1993 fue el valor mínimo alcanzado.',
 'paths': ['pobreza.md']}

In [130]:
path = Path("./frases_originales.json")

with path.open("r", encoding="utf-8") as f:
    data = json.load(f)

In [149]:
res = []

for query in data['frases_originales'][16:]:
  print(query)
  res_query = check_truth(query)
  res.append(res_query)
  print(res_query)

En 2023, solo el 14.5% de los beneficiarios pertenecían al 10% de los individuos más pobres en la distribución del ingreso antes de las transferencias, mientras que 21,8% pertenece al 10% siguiente de la distribución.
{'Resultado': 'falso', 'Respuesta': 'En 2023, el 27,9% de los beneficiarios pertenecían al 10% de los individuos más pobres en la distribución del ingreso antes de las transferencias, según el contexto, lo cual contradice el 14.5% indicado en la sentencia. La cifra del 21,8% para el 10% siguiente sí es correcta.', 'paths': ['pobreza.md']}
Cerca de la mitad de los beneficiarios pertenece al 20% (quintil 1) más pobre de la población.
{'Resultado': 'verdad', 'Respuesta': 'Cerca de la mitad de los beneficiarios pertenece al 20% (quintil 1) más pobre de la población, específicamente, en 2023, el 27,9% pertenecía al 10% más pobre y el 21,8% al 10% siguiente de la distribución, sumando casi la mitad.', 'paths': ['pobreza.md']}
Casi el 40% (39,1%) no cuenta en su vivienda con clo

In [163]:
import json
from pathlib import Path

path = Path("./res.json")  # cambia el nombre si querés

with path.open("w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)

print("Guardado en:", path)

Guardado en: res.json
